In [1]:
# https://blog.paperspace.com/writing-cnns-from-scratch-in-pytorch/
import os
import ast
from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torchvision
import torch.nn.functional as F
import torch
import torch.optim as optim
import helper
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data.sampler import SubsetRandomSampler
# import torch.utils.data as data
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchsummary import summary
from imblearn.under_sampling import RandomUnderSampler

num_classes = 12
learning_rate = 0.001
num_epochs = 3
batch_size = 64
patience = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
ROOT_DIR = "/workspace/resnet/"
ROOT_DATA_DIR = "/workspace/resnet/data/raw"
ARCHIVE_DIR = os.path.join(ROOT_DATA_DIR, "archive.zip")
DATA_DIR = os.path.join(ROOT_DATA_DIR)#, "labelme-12-50k")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TRAIN_ANNOT_PATH = os.path.join(TRAIN_DIR, "annotation.txt")
TEST_DIR = os.path.join(DATA_DIR, "test")
TEST_ANNOT_PATH = os.path.join(TEST_DIR, "annotation.txt")
CLASSES_TXT_PATH = os.path.join(DATA_DIR, "classes.txt")
INTERIM_DATA_DIR = "/workspace/resnet/data/interim/"
MODEL_DIR = os.path.join(ROOT_DIR, "models")
CNN_PATH = os.path.join(MODEL_DIR, "2.0_cnn-balanced")

# import zipfile
# with zipfile.ZipFile(ARCHIVE_DIR, 'r') as zip_ref:
#     zip_ref.extractall(ROOT_DATA_DIR)

cpu


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
train_df_balanced = pd.read_csv(os.path.join(INTERIM_DATA_DIR,"balanced-train-40000.csv"))
test_df = pd.read_csv(os.path.join(INTERIM_DATA_DIR,"test-10000.csv"))

In [4]:
transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5, 0.5, 0.5))]
            )

In [5]:
class MyDataset(Dataset):
    def __init__(self,annot_df,transform = None):
        self.annotations = annot_df
        self.transform = transform
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        img = Image.open(self.annotations.img_path[index])
        y_label = torch.tensor(int(self.annotations.int_label[index]))
        if self.transform:
            image = transform(img)
        else:
            image = img
        return (image,y_label)

In [6]:
# train_data = MyDataset(train_df,transform = transform)
test_data = MyDataset(test_df,transform = transform)
train_data_balanced = MyDataset(train_df_balanced,transform = transform)


# train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
train_loader_balanced = DataLoader(dataset = train_data_balanced, batch_size = batch_size, shuffle = True)

test_loader

In [7]:
train_df_balanced.int_label.value_counts()

0     3438
9     3422
7     3392
6     3325
11    3317
10    3312
8     3305
1     3303
2     3302
4     3297
5     3295
3     3292
Name: int_label, dtype: int64

In [8]:
class CNN(nn.Module):
    def __init__(self,num_classes):
        super(CNN,self).__init__()
        # input channels is 3 because of RGB channels
        #((w-f+2P)/s) + 1
#         self.conv_l1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3)
#         self.conv_l2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3)
        self.conv_l1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3)
        self.conv_l2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
#         self.conv_l3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
#         self.conv_l4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
        self.conv_l3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
        self.conv_l4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_l5 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
        self.conv_l6 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
#         self.conv_l5 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
#         self.conv_l6 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
#         self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(50176,128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128,num_classes)
        
    def forward(self,x):
#         print(x.size())
        out = self.conv_l1(x)
#         print(out.size())
        out = self.conv_l2(out)
#         print(out.size())
        out = self.max_pool1(out)
#         print(out.size())
        out = self.conv_l3(out)
#         print(out.size())
        out = self.conv_l4(out)
#         print(out.size())
        out = self.max_pool2(out)
    
        out = self.conv_l5(out)
#         print(out.size())
        out = self.conv_l6(out)
#         print(out.size())
        out = self.max_pool2(out)
#         print(out.size())
        out = out.reshape(out.size(0), -1)
        
#         print(out.size())
        out = self.fc1(out)
#         print(out.size())
        out = self.relu1(out)
#         print(out.size())
        out = self.fc2(out)
        return out

# layers = [module for module in model.modules()][1:]
# summary(CNN(12),(32, 3, 256, 256))
# layers

model = CNN(num_classes)
print(model)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

# total_step = len(train_loader)
total_step = len(train_loader_balanced)

best_score = None
counter = 0

for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader_balanced):
#     for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    if best_score is None:
        best_score = loss
    else:
            # Check if val_loss improves or not.
        if loss < best_score:
                # val_loss improves, we update the latest best_score, 
                # and save the current model
            best_score = loss
            torch.save(model.state_dict(), CNN_PATH)
        else:
                # val_loss does not improve, we increase the counter, 
                # stop training if it exceeds the amount of patience
            counter += 1
            if counter >= patience:
                break
    if epoch == num_epochs:
        torch.save(model.state_dict(), CNN_PATH)

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


CNN(
  (conv_l1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv_l2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_l3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_l4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_l5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_l6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=12, bias=True)
)
Epoch [1/3], Loss: 2.4907
Epoch [1/3], Loss: 2.4792
Epoch [1/3], Loss: 2.4756
Epoch [1/3], Loss: 2.4930
Epoch [1/3], Loss: 2.4782
Epoch [1/3], Loss: 2.4862
Epoch [1/3], Loss: 2.4819
Epoch [1/3], Loss: 2.

Epoch [1/3], Loss: 2.2488
Epoch [1/3], Loss: 1.7551
Epoch [1/3], Loss: 1.7578
Epoch [1/3], Loss: 2.0122
Epoch [1/3], Loss: 2.0154
Epoch [1/3], Loss: 2.1456
Epoch [1/3], Loss: 2.1931
Epoch [1/3], Loss: 1.8949
Epoch [1/3], Loss: 1.8586
Epoch [1/3], Loss: 2.1078
Epoch [1/3], Loss: 1.7528
Epoch [1/3], Loss: 1.8096
Epoch [1/3], Loss: 1.7895
Epoch [1/3], Loss: 1.9979
Epoch [1/3], Loss: 1.9388
Epoch [1/3], Loss: 1.8985
Epoch [1/3], Loss: 1.8541
Epoch [1/3], Loss: 1.7786
Epoch [1/3], Loss: 2.0900
Epoch [1/3], Loss: 1.8190
Epoch [1/3], Loss: 1.7298
Epoch [1/3], Loss: 1.6675
Epoch [1/3], Loss: 1.7577
Epoch [1/3], Loss: 1.9448
Epoch [1/3], Loss: 1.7860
Epoch [1/3], Loss: 1.9789
Epoch [1/3], Loss: 2.0221
Epoch [1/3], Loss: 1.6371
Epoch [1/3], Loss: 1.9222
Epoch [1/3], Loss: 1.9297
Epoch [1/3], Loss: 1.9478
Epoch [1/3], Loss: 2.0459
Epoch [1/3], Loss: 1.9681
Epoch [1/3], Loss: 1.7442
Epoch [1/3], Loss: 1.8064
Epoch [1/3], Loss: 1.8856
Epoch [1/3], Loss: 1.8590
Epoch [1/3], Loss: 1.8365
Epoch [1/3],

Epoch [1/3], Loss: 1.2574
Epoch [1/3], Loss: 1.4609
Epoch [1/3], Loss: 1.5993
Epoch [1/3], Loss: 1.7880
Epoch [1/3], Loss: 1.4054
Epoch [1/3], Loss: 1.3235
Epoch [1/3], Loss: 1.3823
Epoch [1/3], Loss: 1.1258
Epoch [1/3], Loss: 1.3052
Epoch [1/3], Loss: 1.3033
Epoch [1/3], Loss: 1.4108
Epoch [1/3], Loss: 1.3003
Epoch [1/3], Loss: 1.3552
Epoch [1/3], Loss: 1.2758
Epoch [1/3], Loss: 1.4978
Epoch [1/3], Loss: 1.6638
Epoch [1/3], Loss: 1.4251
Epoch [1/3], Loss: 1.3808
Epoch [1/3], Loss: 1.3840
Epoch [1/3], Loss: 1.4109
Epoch [1/3], Loss: 1.3898
Epoch [1/3], Loss: 1.4427
Epoch [1/3], Loss: 1.7481
Epoch [1/3], Loss: 1.1206
Epoch [1/3], Loss: 1.1206
Epoch [2/3], Loss: 1.3294
Epoch [2/3], Loss: 1.3862
Epoch [2/3], Loss: 1.2846
Epoch [2/3], Loss: 1.5280
Epoch [2/3], Loss: 0.9812
Epoch [2/3], Loss: 1.2763
Epoch [2/3], Loss: 1.3720
Epoch [2/3], Loss: 1.3451
Epoch [2/3], Loss: 1.8149
Epoch [2/3], Loss: 1.4851
Epoch [2/3], Loss: 1.3395
Epoch [2/3], Loss: 1.1345
Epoch [2/3], Loss: 1.3150
Epoch [2/3],

Epoch [2/3], Loss: 0.8717
Epoch [2/3], Loss: 0.8465
Epoch [2/3], Loss: 0.8088
Epoch [2/3], Loss: 1.0987
Epoch [2/3], Loss: 0.8607
Epoch [2/3], Loss: 1.1171
Epoch [2/3], Loss: 0.9617
Epoch [2/3], Loss: 0.7910
Epoch [2/3], Loss: 1.0981
Epoch [2/3], Loss: 0.8484
Epoch [2/3], Loss: 1.0994
Epoch [2/3], Loss: 0.9937
Epoch [2/3], Loss: 1.0425
Epoch [2/3], Loss: 1.0136
Epoch [2/3], Loss: 0.8066
Epoch [2/3], Loss: 1.0590
Epoch [2/3], Loss: 1.0621
Epoch [2/3], Loss: 0.8218
Epoch [2/3], Loss: 0.8358
Epoch [2/3], Loss: 0.7269
Epoch [2/3], Loss: 1.1048
Epoch [2/3], Loss: 0.9721
Epoch [2/3], Loss: 1.1078
Epoch [2/3], Loss: 1.2035
Epoch [2/3], Loss: 0.9647
Epoch [2/3], Loss: 1.0213
Epoch [2/3], Loss: 1.0115
Epoch [2/3], Loss: 0.7829
Epoch [2/3], Loss: 1.0889
Epoch [2/3], Loss: 0.8552
Epoch [2/3], Loss: 1.1361
Epoch [2/3], Loss: 0.8599
Epoch [2/3], Loss: 0.9533
Epoch [2/3], Loss: 1.1089
Epoch [2/3], Loss: 0.9056
Epoch [2/3], Loss: 0.9646
Epoch [2/3], Loss: 0.7646
Epoch [2/3], Loss: 0.6326
Epoch [2/3],

Epoch [2/3], Loss: 0.7034
Epoch [2/3], Loss: 0.5000
Epoch [2/3], Loss: 0.6171
Epoch [2/3], Loss: 0.6355
Epoch [2/3], Loss: 0.7399
Epoch [2/3], Loss: 0.7825
Epoch [2/3], Loss: 0.6232
Epoch [2/3], Loss: 0.8022
Epoch [2/3], Loss: 0.8407
Epoch [2/3], Loss: 0.8550
Epoch [2/3], Loss: 0.8065
Epoch [2/3], Loss: 0.6835
Epoch [2/3], Loss: 0.4203
Epoch [2/3], Loss: 0.7389
Epoch [2/3], Loss: 0.6444
Epoch [2/3], Loss: 0.8570
Epoch [2/3], Loss: 0.7228
Epoch [2/3], Loss: 0.5136
Epoch [2/3], Loss: 0.5136
Epoch [3/3], Loss: 0.6355
Epoch [3/3], Loss: 0.4734
Epoch [3/3], Loss: 0.3853
Epoch [3/3], Loss: 0.5041
Epoch [3/3], Loss: 0.6442
Epoch [3/3], Loss: 0.4998
Epoch [3/3], Loss: 0.6934
Epoch [3/3], Loss: 0.4197
Epoch [3/3], Loss: 0.3526
Epoch [3/3], Loss: 0.5827
Epoch [3/3], Loss: 0.3365
Epoch [3/3], Loss: 0.5989
Epoch [3/3], Loss: 0.4228
Epoch [3/3], Loss: 0.4631
Epoch [3/3], Loss: 0.4899
Epoch [3/3], Loss: 0.6022
Epoch [3/3], Loss: 0.3915
Epoch [3/3], Loss: 0.3798
Epoch [3/3], Loss: 0.4204
Epoch [3/3],

Epoch [3/3], Loss: 0.4831
Epoch [3/3], Loss: 0.2441
Epoch [3/3], Loss: 0.3473
Epoch [3/3], Loss: 0.2800
Epoch [3/3], Loss: 0.3866
Epoch [3/3], Loss: 0.5666
Epoch [3/3], Loss: 0.7406
Epoch [3/3], Loss: 0.3660
Epoch [3/3], Loss: 0.3136
Epoch [3/3], Loss: 0.5649
Epoch [3/3], Loss: 0.2077
Epoch [3/3], Loss: 0.4460
Epoch [3/3], Loss: 0.3808
Epoch [3/3], Loss: 0.4216
Epoch [3/3], Loss: 0.5039
Epoch [3/3], Loss: 0.2074
Epoch [3/3], Loss: 0.6599
Epoch [3/3], Loss: 0.5069
Epoch [3/3], Loss: 0.8395
Epoch [3/3], Loss: 0.6171
Epoch [3/3], Loss: 0.3317
Epoch [3/3], Loss: 0.3538
Epoch [3/3], Loss: 0.4392
Epoch [3/3], Loss: 0.3826
Epoch [3/3], Loss: 0.4091
Epoch [3/3], Loss: 0.4582
Epoch [3/3], Loss: 0.3464
Epoch [3/3], Loss: 0.4451
Epoch [3/3], Loss: 0.4848
Epoch [3/3], Loss: 0.4396
Epoch [3/3], Loss: 0.3729
Epoch [3/3], Loss: 0.5542
Epoch [3/3], Loss: 0.5658
Epoch [3/3], Loss: 0.5388
Epoch [3/3], Loss: 0.3123
Epoch [3/3], Loss: 0.6193
Epoch [3/3], Loss: 0.3108
Epoch [3/3], Loss: 0.5457
Epoch [3/3],

Epoch [3/3], Loss: 0.2800
Epoch [3/3], Loss: 0.4288
Epoch [3/3], Loss: 0.5011
Epoch [3/3], Loss: 0.4378
Epoch [3/3], Loss: 0.3617
Epoch [3/3], Loss: 0.4988
Epoch [3/3], Loss: 0.5420
Epoch [3/3], Loss: 0.4223
Epoch [3/3], Loss: 0.3322
Epoch [3/3], Loss: 0.3306
Epoch [3/3], Loss: 0.4033
Epoch [3/3], Loss: 0.3711
Epoch [3/3], Loss: 0.3711


In [9]:
model.eval()

CNN(
  (conv_l1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv_l2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_l3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_l4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_l5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_l6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=12, bias=True)
)

In [12]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader_balanced:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))




Accuracy of the network on the 40000 train images: 94.08 %


In [11]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))




Accuracy of the network on the 10000 test images: 56.21 %


In [ ]:
summary(CNN(12),(3, 256, 256))

In [ ]:
images, labels = next(iter(train_loader))
images.shape

In [ ]:
images[0].shape

In [ ]:
labels.shape

In [ ]:
torchvision.transforms.functional.get_image_size(images)

In [ ]:
print(test_df.str_label[1500])
Image.open(test_df.img_path[1500])
print(train_df.str_label[1000],train_df.img_path[1000])
Image.open(train_df.img_path[1000])

In [ ]:
print(train_df.str_label[1000],train_df.img_path[1000])
Image.open(train_df.img_path[1000])

In [ ]:
# transform = transforms.Compose(
#         [transforms.ToTensor(),
#         transforms.Normalize((0.5,0.5,0.5),(0.5, 0.5, 0.5))]
#     )

# test_data = datasets.ImageFolder('/workspace/resnet/data/raw/test/', transform=transform)
# test_data_loader  = data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4) 

# train_data = datasets.ImageFolder('/workspace/resnet/data/raw/train/', transform=transform)
# train_data_loader  = data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4) 

# train_data